# 배포 생성하기

모듈 5에서 만들었던 `task_maistro` 앱의 배포를 생성해 보겠습니다.

## 코드 구조

LangGraph Platform 배포를 생성하려면 [다음과 같은 정보가 제공되어야 합니다](https://langchain-ai.github.io/langgraph/concepts/application_structure/).

* [LangGraph API 설정 파일](https://langchain-ai.github.io/langgraph/concepts/application_structure/#configuration-file) - `langgraph.json`
* 애플리케이션의 로직을 구현하는 그래프 - 예: `task_maistro.py`
* 애플리케이션 실행에 필요한 의존성을 명시하는 파일 - `requirements.txt`
* 애플리케이션 실행에 필요한 환경 변수 제공 - `.env` 또는 `docker-compose.yml`

이 파일들은 이미 `module-6/deployment` 디렉토리에 준비되어 있습니다!

## CLI

[LangGraph CLI](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/)는 LangGraph Platform 배포 생성을 위한 커맨드 라인 인터페이스(CLI)입니다.

In [1]:
%%capture --no-stderr
%pip install -U langgraph-cli

[자체 호스팅 배포(self-hosted deployment)](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#how-to-do-a-self-hosted-deployment-of-langgraph)를 생성하기 위해 몇 가지 단계를 따르겠습니다.

### LangGraph 서버를 위한 Docker 이미지 빌드

먼저 `langgraph` CLI를 사용하여 [LangGraph 서버](https://docs.google.com/presentation/d/18MwIaNR2m4Oba6roK_2VQcBE_8Jq_SI7VHTXJdl7raU/edit#slide=id.g313fb160676_0_32)를 위한 Docker 이미지를 생성합니다.

이 과정은 우리가 만든 그래프와 의존성 파일들을 Docker 이미지로 패키징합니다.

Docker 이미지는 애플리케이션 실행에 필요한 코드와 의존성을 포함하는 Docker 컨테이너용 템플릿입니다.

[Docker](https://docs.docker.com/engine/install/)가 설치되어 있는지 확인한 후, 다음 명령어를 실행하여 `my-image`라는 Docker 이미지를 생성하세요:

```shell
$ cd module-6/deployment
$ langgraph build -t my-image
```

### Redis와 PostgreSQL 설정

만약 이미 Redis와 PostgreSQL을 실행 중이라면(예: 로컬 또는 다른 서버), Redis와 PostgreSQL의 URI를 직접 지정하여 LangGraph 서버 컨테이너를 [단독으로](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#running-the-application-locally) 생성하고 실행할 수 있습니다:

```shell
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="여기에_Redis_URI_입력" \
    -e DATABASE_URI="여기에_PostgreSQL_URI_입력" \
    -e LANGSMITH_API_KEY="여기에_LangSmith_API_키_입력" \
    my-image
```

또는, 제공된 `docker-compose.yml` 파일을 사용하여 정의된 서비스에 따라 세 개의 개별 컨테이너를 생성할 수도 있습니다:

  * `langgraph-redis`: 공식 Redis 이미지를 사용하여 새 컨테이너를 생성합니다.
  * `langgraph-postgres`: 공식 Postgres 이미지를 사용하여 새 컨테이너를 생성합니다.
  * `langgraph-api`: 미리 빌드한 여러분의 이미지를 사용하여 새 컨테이너를 생성합니다.

간단히 `docker-compose-example.yml` 파일을 복사한 후, 배포된 `task_maistro` 앱을 실행하기 위해 다음 환경 변수들을 추가하세요:

  * `IMAGE_NAME` (예: `my-image`)
  * `LANGSMITH_API_KEY`
  * `OPENAI_API_KEY`

그런 다음, [배포를 시작](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#using-docker-compose)하세요:

```shell
$ cd module-6/deployment
$ docker compose up
```